In [ ]:
import gf

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import imageio as iio
import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200


In [ ]:
def to_image(im):
    plt.axis('off')
    if len(im.shape)>2:
        imshow(im)
    else:
        imshow(im, cmap='gray')
    plt.show()

## Image Fusion with Guided Filtering 

### Guided filter

In [ ]:
im = iio.imread('../data/petrovic/fused012_1.tif')/255
to_image(im)

In [ ]:
from gf.filters import guided_filter

GF with grey input / grey guide

In [ ]:
gf = guided_filter(im, im, 5, 0.1)
to_image(gf)

In [ ]:
im2 = iio.imread('../data/lytro/lytro-01-A.jpg')/255
to_image(im2)

GF with rgb input / grey guide

In [ ]:
gf = guided_filter(im2, im2[:,:,0], 5,0.05)
to_image(gf)

GF with rgb input / rgb guide

In [ ]:
gf = guided_filter(im2, im2, 1,0.05)
to_image(gf)

In [ ]:
from gf.fusion import gff
mygff = gff([im2])

### Decomposition

In [ ]:
b, d = mygff.decompose(im2)
to_image(b)

In [ ]:
md, Md = np.min(d), np.max(d)
to_image((d-md)/(Md-md))

### Salicency map

In [ ]:
images_paths = ['../data/lytro/lytro-01-'+x+'.jpg' for x in ['A', 'B']]
ims = [iio.imread(path)/255 for path in images_paths]
for im in ims:
    to_image(im)
mygff = gff(ims)

In [ ]:
for im in ims:
    to_image(mygff.saliency(im))

### Weights

In [ ]:
weights = mygff.weight_maps(ims)
to_image(weights[0])
to_image(weights[1])

### Fusion

In [ ]:
f = mygff.fusion()
to_image(f)

## Experiments

### Guided filter vs other filters

bilateral filter

In [ ]:
from skimage.restoration import denoise_bilateral
def mybilateral(p, i, r, eps):
    return denoise_bilateral(p)

In [ ]:
f = mygff.fusion(filt=mybilateral)
to_image(f)

pas si mal... mais voir ce qu'il se passe avec l'HDR

### No separation base/details

In [ ]:
f_ns = mygff.fusion_without_separation()
to_image(f_ns)

Le bout du club est plus flou ! Plus facile de voir ce qu'il se passe avec l'exposure fusion ?

### Exposure fusion

In [ ]:
images_paths = ['../data/MEFDatabase/source/Memorial_Debevec97/memorial00'+str(x)+'.png' 
                for x in range(61,77)]
ims = [iio.imread(path)/255 for path in images_paths]
mygff = gff(ims)

In [ ]:
f = mygff.fusion()
to_image(f)

with bilateral filter:

In [ ]:
f = mygff.fusion(filt=mybilateral)
to_image(f)

without separation base/details:

In [ ]:
for r in [7,45]:
    for eps in [1e-6, 0.3]:
        print(f'r={r} and eps={eps}')
        f = mygff.fusion_without_separation(r=r, eps=eps)
        to_image(f)

r petit -> patches

epsilon petit -> + homogène ?

### Other color spaces : HSV

In [ ]:
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb

In [ ]:
images_paths = ['../data/lytro/lytro-01-'+x+'.jpg' for x in ['A', 'B']]
ims = [rgb_to_hsv(iio.imread(path)/255) for path in images_paths]
mygff = gff(ims, color='hsv')
f = mygff.fusion()
to_image(hsv_to_rgb(f))

En adaptant le filtre, cela fontionne aussi en HSV malgré les discontinuités dans le channel H (la teinte), dont les valeurs sont cycliques (0=1).

In [ ]:
im2 = iio.imread('../data/lytro/lytro-01-A.jpg')/255
im2_hsv = rgb_to_hsv(im2)
h, s, v = im2_hsv[:,:,0], im2_hsv[:,:,1], im2_hsv[:,:,2]
to_image(h)